In [288]:
import numpy as np
import pandas as pd
import statistics
import math
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns

In [289]:
pd.set_option('display.max_colwidth', -1)

In [290]:
train = pd.read_csv("data/train_limpio.csv",encoding = "ISO-8859-1")
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [291]:
df = train.iloc[1:5]
df.head()

,id,keyword,location,text,target
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


# Las palabras mas comunes de las noticias y su correspondiente target

In [292]:
from itertools import chain

In [293]:
lens = train['text'].str.split(' ').map(len)
lens

0       13
1       7 
2       22
3       9 
4       17
        ..
6984    13
6985    20
6986    19
6987    26
6988    20
Name: text, Length: 6989, dtype: int64

In [294]:
res = pd.DataFrame({'id': np.repeat(train['id'], lens),
                    'keyword': np.repeat(train['keyword'], lens),
                    'location': np.repeat(train['location'], lens),
                    'text': np.repeat(train['text'], lens),
                    'target': np.repeat(train['target'], lens)
                   }).reset_index()

In [295]:
res.head(60)

,index,id,keyword,location,text,target
0,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
2,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
3,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
4,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
5,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
6,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
7,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
8,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
9,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1


In [296]:
len(res)

104199

In [297]:
id_actual = 0
posicion_actual = 0
palabras = 0
for i in range (len(res)):
    current_id = res.loc[i,'id']
    if(id_actual !=current_id):
        posicion_actual = 0
        palabras = list((res.loc[i,'text'].split(' ')))
        id_actual = current_id
    else:
        posicion_actual = posicion_actual + 1
    res.loc[i,'text'] = palabras[posicion_actual]

In [300]:
res.head(100)

,index,id,keyword,location,text,target
0,0,1,NaN,NaN,Our,1
1,0,1,NaN,NaN,Deeds,1
2,0,1,NaN,NaN,are,1
3,0,1,NaN,NaN,the,1
4,0,1,NaN,NaN,Reason,1
5,0,1,NaN,NaN,of,1
6,0,1,NaN,NaN,this,1
7,0,1,NaN,NaN,#earthquake,1
8,0,1,NaN,NaN,May,1
9,0,1,NaN,NaN,ALLAH,1


In [306]:
res['text'] = res['text'].map(lambda x: x.lstrip('#').rstrip(',')).map(lambda x: (x.lower()))

In [308]:
res.head(100)

,index,id,keyword,location,text,target
0,0,1,NaN,NaN,our,1
1,0,1,NaN,NaN,deeds,1
2,0,1,NaN,NaN,are,1
3,0,1,NaN,NaN,the,1
4,0,1,NaN,NaN,reason,1
5,0,1,NaN,NaN,of,1
6,0,1,NaN,NaN,this,1
7,0,1,NaN,NaN,earthquake,1
8,0,1,NaN,NaN,may,1
9,0,1,NaN,NaN,allah,1


In [309]:
res.to_csv('data/train_una_palabra_por_fila.csv', index=False)